In [85]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from dataclasses import dataclass

In [86]:
#initial settings
PUZZLE_DIM = 3
#action = namedtuple('Action', ['pos1', 'pos2'])
SOLUTION = np.array([n+1 for n in range(PUZZLE_DIM*PUZZLE_DIM)])
SOLUTION[-1] = 0
puzzle = SOLUTION[:]
print(SOLUTION)

[1 2 3 4 5 6 7 8 0]


In [87]:
#utility functions and structures

#action class

@dataclass
class State:
    board: np.array
    gn: int = 0
    fn: int = np.inf
    priority: int = np.inf
    __eq__ = lambda self, other: (np.array_equal(self.board, other.board) and self.gn == other.gn)
    
@dataclass
class Action:
    x1: int
    y1: int
    x2: int
    y2: int
    #gn: int = 0
    #fn: int = 0

def print_board(state: State):
    state = state.board
    matrix = np.resize(state, (PUZZLE_DIM, PUZZLE_DIM))
    print(matrix)

In [88]:
def available_actions(state: State) -> list['Action']:
    #find the empty tile
    board = state.board
    board = np.resize(board, (PUZZLE_DIM, PUZZLE_DIM))
    ii = np.where(board == 0)
    x= ii[0][0]
    y= ii[1][0]
    #print(x,y)
    #print_board (state)
    actions = list()
    if x > 0:
        actions.append(Action(x, y, x - 1, y))
    if x < PUZZLE_DIM - 1:
        actions.append(Action(x, y, x + 1, y))
    if y > 0:
        actions.append(Action(x, y, x, y - 1))
    if y < PUZZLE_DIM - 1:
        actions.append(Action(x, y, x, y + 1))
    return actions



def do_action(state: State, action: 'Action') -> State:
    new_board = state.board.copy()
    new_board[action.x1, action.y1], new_board[action.x2, action.y2] = new_board[action.x2, action.y2], new_board[action.x1, action.y1]
    new_state = State(new_board, state.gn + 1, 0)
    #print_board(new_state)
    return new_state

In [89]:
RANDOMIZE_STEPS = 100_000
state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)),0,0)
#print_board(state)
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
print_board(state)

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

[[2 4 0]
 [1 7 3]
 [8 5 6]]


In [90]:
def compute_hcost_manhattan(state: State) -> int:
    board = state.board
    board = np.resize(board, (PUZZLE_DIM, PUZZLE_DIM))
    distance = 0
    solution = np.resize(SOLUTION, (PUZZLE_DIM, PUZZLE_DIM))
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if board[i, j] == 0:
                continue
            ii = np.where(solution == board[i, j])
            x= ii[0][0]
            y= ii[1][0]
            x, y = divmod(board[i, j] - 1, PUZZLE_DIM)
            distance += abs(x - i) + abs(y - j)
    return distance

compute_hcost_manhattan(state)

np.int64(10)

In [91]:
def A_star_search(initial_state: State):
    
    visited = []
    total_cost=1
    frontier = [initial_state]
    frontier[0].fn = compute_hcost_manhattan(initial_state)
    #frontier[0].priority = 0

    i=0
    score = 0
    while frontier:
        #SOSTITUISCI CON UNA PRIORITY QUEUE O CON UN MIN HEAP
        current = min(frontier, key=lambda x: x.priority)
        if np.array_equal(current.board, SOLUTION):
            print("SOLUTION FOUND")
            return current, total_cost
        
        frontier.remove(current)
        visited.append(current)
        for action in available_actions(current):
            new_state = do_action(current, action)
            #gn è implicitamente incrementato in do_action
            new_state.fn = compute_hcost_manhattan(new_state)
            #non capisco se fare questo nel mio caso abbia senso -> forse posso prendermi direttamente new_state.gn
            new_cost = current.gn + 1
            isVisited = False
            for closed_state in visited:
                #non ho capito se questo ha senso
                if np.array_equal(closed_state.board, new_state.board):
                    isVisited = True
                    break
            if not isVisited or new_cost < new_state.gn:
                new_state.gn = new_cost
                new_state.priority = new_state.gn + new_state.fn
                frontier.append(new_state)           
        total_cost+=1
        i+=1
        if i%300==0:
            print(i)
            print("frontier size: ", len(frontier))
            print_board(current)
    return current, total_cost

solution, cost = A_star_search(state)
print_board(solution)

300
frontier size:  200
[[4 1 3]
 [0 5 7]
 [2 8 6]]
600
frontier size:  386
[[1 3 0]
 [5 4 2]
 [7 8 6]]
900
frontier size:  558
[[1 5 2]
 [0 4 3]
 [7 8 6]]
1200
frontier size:  770
[[7 4 3]
 [2 8 5]
 [1 0 6]]
1500
frontier size:  956
[[1 2 4]
 [5 6 8]
 [7 0 3]]
1800
frontier size:  1116
[[1 2 7]
 [0 3 6]
 [8 4 5]]
2100
frontier size:  1244
[[7 1 4]
 [0 2 5]
 [8 6 3]]
2400
frontier size:  1446
[[2 5 3]
 [1 0 6]
 [4 7 8]]
2700
frontier size:  1669
[[4 2 3]
 [8 7 1]
 [0 5 6]]
3000
frontier size:  1843
[[7 2 3]
 [1 4 5]
 [6 0 8]]
3300
frontier size:  2048
[[1 2 3]
 [4 8 6]
 [5 0 7]]
3600
frontier size:  2196
[[2 0 4]
 [6 7 5]
 [1 8 3]]
3900
frontier size:  2362
[[7 3 1]
 [2 4 6]
 [0 8 5]]
4200
frontier size:  2466
[[5 2 3]
 [1 6 4]
 [8 7 0]]
4500
frontier size:  2557
[[7 2 3]
 [5 4 6]
 [1 8 0]]
4800
frontier size:  2716
[[1 3 6]
 [0 7 2]
 [4 8 5]]
5100
frontier size:  2909
[[4 3 7]
 [0 5 6]
 [2 1 8]]
5400
frontier size:  3096
[[4 2 3]
 [5 8 1]
 [0 7 6]]
5700
frontier size:  3331
[[2 1 3]
 

KeyboardInterrupt: 